In [1]:
import sys
import json
from pprint import pprint as print_
from collections import OrderedDict

sys.path.append("../src/")

import numpy as np
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [2]:
roleset44 = {'содержание высказывания',
 'говорящий',
 'субъект социального отношения',
 'субъект психологического состояния',
 'содержание действия',
 'агенс',
 'тема',
 'конечная точка',
 'сфера',
 'контрагент',
 'субъект перемещения',
 'причина',
 'субъект поведения',
 'ситуация в фокусе',
 'исходный посессор',
 'субъект физиологической реакции',
 'адресат',
 'пациенс',
 'срок',
 'источник звука',
 'место',
 'признак',
 'потенциальная угроза',
 'субъект ментального состояния',
 'конечный посессор',
 'результат',
 'стимул',
 'субъект восприятия',
 'эффектор',
 'траектория',
 'содержание мысли',
 'пациенс перемещения',
 'каузатор',
 'предмет высказывания',
 'начальная точка',
 'способ',
 'пациенс социального отношения',
 'статус',
 'предмет мысли',
 'цель',
 'потенциальный пациенс',
 'контрагент социального отношения',
 'эталон',
 'признак действия'}

In [3]:
ppl = PipelineDefault(address_morph=('vmh1.isa.ru', 3300),
                      address_syntax=('vmh1.isa.ru', 3311),
                      address_srl=('vmh1.isa.ru', 3322))

In [4]:
def get_roles_pred(lemma, role_annot, part_id):
    ann_sent = role_annot[part_id]
    predicates = {}
    arguments = {}
    for event in ann_sent:
        predicate = {
            'lemma': lemma[part_id][event.pred[0]],
        }
        predicates[event.pred[0]] = predicate
        arguments[event.pred[0]] = []
        for arg in event.args:
            #print(vars(arg))
            argument = {
                'tag': arg.tag,
                'lemma': lemma[part_id][arg.begin]
            }
            arguments[event.pred[0]].append(argument)
            
    return predicates, arguments

In [5]:
def get_example(corpus, ex_number, part_id):
    words = []
    for obj in corpus[ex_number][1][part_id]:
        words.append(obj['form'])
        
    return " ".join(words)

def get_roles_true(corpus, ex_number, part_id):
    predicates = {}
    arguments = {}
    for i, obj in enumerate(corpus[ex_number][1][part_id]):
        if 'rank' in obj:
            if obj['rank'] == 'Предикат':
                predicate = {
                    'lemma': obj['lemma']
                }
                predicates[i] = predicate
            else:
                argument = {
                    'lemma': obj['lemma'],
                    'tag': obj['rolepred1']
                }
                pred_id = obj['fillpred']
                if pred_id not in arguments.keys():
                    arguments[pred_id] = []
                arguments[pred_id].append(argument)
    return predicates, arguments

In [6]:
with open("../data/cleared_corpus.json", 'r', encoding='utf-8') as f:
    corpus = json.load(f)

In [77]:
def random_predictions(corpus, ppl, n_samples=100):
    samples_idxs = np.random.choice(len(corpus), size=n_samples)
    true_roles = [get_roles_true(corpus, ex_num, 0) for ex_num in samples_idxs]
    texts = [get_example(corpus, ex_num, 0) for ex_num in samples_idxs]
    pred_roles = [ppl(text) for text in tqdm(texts, desc='Analyzing texts')]
    pred_roles = [get_roles_pred(res['lemma'], res['srl'], 0) for res in pred_roles]
    
    repl_roles = {
        'агенс - субъект восприятия' : 'субъект восприятия',
        'агенс - субъект ментального состояния' : 'субъект ментального состояния',
        'результат / цель' : 'результат',
        'место - пациенс' : 'место',
        'говорящий - субъект психологического состояния' : 'субъект психологического состояния'
    }
    for role, val in repl_roles.items():
        for pair in true_roles:
            for _, args in pair[1].items():
                for arg in args:
                    arg['tag'] = arg['tag'].replace(role, val)
    
    return true_roles, pred_roles, texts

In [99]:
true_roles, pred_roles, texts = random_predictions(corpus, ppl, n_samples=2000)

KeyError: 'lemma'

In [96]:
def compute_metrics(y_pred, y_true, report_to=sys.stdout, roleset=roleset44):

    true_positive = 0
    condition_positive = 0
    predicted_condition_positive = 0
    error_examples = []
    
    print_func = lambda x: print(x, file=report_to)
    
    for i, (true_predicates, true_arguments) in enumerate(y_true):
        print_func(f"Inspecting example {i}")
        print_func(f"Expecting true predicates {true_predicates}")
        print_func(f"Expecting true arguments  {true_arguments}")
        
        pred_predicates, pred_arguments = y_pred[i]
        print_func(f"Got predicted predicates  {pred_predicates}")
        print_func(f"Got predicted arguments   {pred_arguments}")
        
        print_func("-"*60)
        
        for true_pred_idx, true_predicate in true_predicates.items():
            if true_pred_idx in pred_predicates:
                print_func(f"Matched predicate {true_pred_idx} = {true_predicate}")
                
                true = true_arguments[true_pred_idx]
                pred_arguments_i = pred_arguments[true_pred_idx]
                
                true_arguments_i = []
                
                for idx, true_argument in enumerate(true):
                    if true_argument['tag'] in roleset:
                        true_arguments_i.append(true[idx])
                        
                print_func(f"Expecting arguments  {true_arguments_i}")
                print_func(f"Got predicted        {pred_arguments_i}")
                print_func(f"Predicted Condition Positive = {len(pred_arguments_i)}")
                print_func(f"Condition Positive           = {len(true_arguments_i)}")
                condition_positive += len(true_arguments_i)
                condition_positive_i = len(true_arguments_i)
                predicted_condition_positive += len(pred_arguments_i)
                
                true_positive_i = 0
                
                error_report = {
                    'example_idx' : i,
                    'predicate': true_predicate,
                    'true_arguments' : true_arguments_i,
                    'predicted_arguments': pred_arguments_i
                }
                
                for j, obj in enumerate(true_arguments_i):
                    true_tag = obj['tag']
                    true_lemma = obj['lemma']
                    for obj_pred in pred_arguments_i:
                        if obj_pred['tag'] == true_tag and obj_pred['lemma'] == true_lemma:
                            true_positive_i += 1
                            
                print_func(f"True Positive = {true_positive_i}")
                if true_positive_i != condition_positive_i:
                    error_examples.append(error_report)
                
                true_positive += true_positive_i
                
        print_func("="*60)
                                
    recall = true_positive/condition_positive  
    percision = true_positive/predicted_condition_positive
    
    return {
        'recall': recall,
        'percision': percision,
        'f1': 2 * ((percision*recall)/(percision+recall)),
        'errors': error_examples
    }

In [97]:
results = compute_metrics(y_pred=pred_roles, y_true=true_roles)

Inspecting example 0
Expecting true predicates {}
Expecting true arguments  {}
Got predicted predicates  {10: {'lemma': 'становиться'}, 22: {'lemma': 'поговорить'}}
Got predicted arguments   {10: [{'tag': 'агенс', 'lemma': 'мы'}, {'tag': 'место', 'lemma': 'дом'}, {'tag': 'пациенс', 'lemma': 'дядя'}], 22: [{'tag': 'контрагент', 'lemma': 'марина'}]}
------------------------------------------------------------
Inspecting example 1
Expecting true predicates {}
Expecting true arguments  {}
Got predicted predicates  {}
Got predicted arguments   {}
------------------------------------------------------------
Inspecting example 2
Expecting true predicates {12: {'lemma': 'вывести'}}
Expecting true arguments  {12: [{'lemma': 'человек', 'tag': 'пациенс перемещения'}, {'lemma': 'рельс', 'tag': 'конечная точка'}]}
Got predicted predicates  {7: {'lemma': 'вспоминать'}, 12: {'lemma': 'выводить'}, 18: {'lemma': 'скидывать'}}
Got predicted arguments   {7: [{'tag': 'тема', 'lemma': 'раз'}, {'tag': 'пред

In [98]:
print_(results)

{'errors': [{'example_idx': 2,
             'predicate': {'lemma': 'вывести'},
             'predicted_arguments': [{'lemma': 'человек', 'tag': 'пациенс'},
                                     {'lemma': 'рельс',
                                      'tag': 'конечная точка'}],
             'true_arguments': [{'lemma': 'человек',
                                 'tag': 'пациенс перемещения'},
                                {'lemma': 'рельс', 'tag': 'конечная точка'}]},
            {'example_idx': 5,
             'predicate': {'lemma': 'миновать'},
             'predicted_arguments': [{'lemma': 'душ', 'tag': 'место'}],
             'true_arguments': [{'lemma': 'душ', 'tag': 'тема'}]},
            {'example_idx': 17,
             'predicate': {'lemma': 'волновать'},
             'predicted_arguments': [{'lemma': 'мир', 'tag': 'причина'},
                                     {'lemma': 'я', 'tag': 'признак'},
                                     {'lemma': 'что-то',
                         